# Read grid function

In [1]:
import pandas as pd
import xarray as xr

In [2]:
hgrid = '/Users/brey/GLOBAL/SCHISM_GLOBAL/hgrid.gr3'

In [59]:
hgrid = '/Users/brey/SCHISM/test/hgrid.gr3'

In [3]:
#extract number of elements, number of nodes
ne,nn = pd.read_csv(hgrid,header=0,skiprows=1,nrows=0,delim_whitespace=True)
    
ne=int(ne)
nn=int(nn)        

ne, nn

(866550, 433277)

In [4]:
#read lon,lat,depth for all nodes
q=pd.read_csv(hgrid,skiprows=2,header=None,delim_whitespace=True,engine='python',nrows=nn,names=['id','x','y','z'])        
q=q.set_index(['id'])
q.head()    



,x,y,z
id,,,
1,128.364768,89.977557,4112
2,85.836638,-89.867898,-2775
3,0.025946,0.076158,4933
4,90.031239,-0.020937,4197
5,179.940933,0.110820,5241


In [5]:
#read connectivity
e = pd.read_csv(hgrid,skiprows=nn+2,header=None,delim_whitespace=True,engine='python',nrows=ne,names=['id','nv','a','b','c'])
e=e.set_index(['id'])
e.head()    



,nv,a,b,c
id,,,,
1,3,74458,76761,263429
2,3,386250,311077,410897
3,3,425016,375158,427834
4,3,65902,65903,65904
5,3,230673,191175,372416


In [6]:
#Number of open boundaries
nob = pd.read_csv(hgrid,header=None,skiprows=ne+nn+2,nrows=1,delimiter='=')[0]
nob

0    0
Name: 0, dtype: int64

In [26]:
if nob.values > 0:
    #Total number of open boundary nodes
    ton = pd.read_csv(hgrid,header=None,skiprows=ne+nn+3,nrows=1,delimiter='=')[0]

    ns = ne + nn + 2 + 2
    nnob = pd.read_csv(hgrid,header=None,skiprows=ns,nrows=1,delimiter='=')[0].values[0]
    label = ' '.join(pd.read_csv(hgrid,header=None,skiprows=ns,nrows=1,delimiter='=')[1].str.split()[0][-3:])
    op = pd.read_csv(hgrid,header=None,skiprows=ns+1,nrows=nnob,names=[label])
    for i in range(1, nob):
        ns = ns + 1 + nnob
        nnob = pd.read_csv(hgrid,header=None,skiprows=ns,nrows=1,delimiter='=')[0].values[0]
        label = ' '.join( pd.read_csv(hgrid,header=None,skiprows=ns,nrows=1,delimiter='=')[1].str.split()[0][-3:] )
        op = pd.concat([op,pd.read_csv(hgrid,header=None,skiprows=ns+1,nrows=nnob,names=[label])], axis=1)

    op.index.name = 'id' # set index name to match q,e
    op.index = op.index + 1 #  ''
else:
    ns = ne + nn + 3
    nnob = 0
    op = pd.DataFrame({})

In [28]:
op.head()

""


In [15]:
#Number of land boundaries
nlb = pd.read_csv(hgrid,header=None,skiprows=ns + 1 + nnob,nrows=1,delimiter='=')[0]
nlb

0    0
Name: 0, dtype: int64

In [32]:
if nlb.values > 0:
    #Total number of land boundary nodes
    tln = pd.read_csv(hgrid,header=None,skiprows=ns + 1 + nnob + 1,nrows=1,delimiter='=')[0]
     
    ns = ns + 1 + nnob + 2
    nnlb, btype = pd.read_csv(hgrid,header=None,skiprows=ns,nrows=1,delimiter='=')[0].str.split()[0]
    nnlb = int(nnlb)
    label = ' '.join(pd.read_csv(hgrid,header=None,skiprows=ns,nrows=1,delimiter='=')[1].str.split()[0][-3:])
    bt=[btype]
    lp = pd.read_csv(hgrid,header=None,skiprows=ns+1,nrows=nnlb,names=[label])
    for i in range(1, nlb):
        ns = ns + 1 + nnlb
        nnlb, btype  = pd.read_csv(hgrid,header=None,skiprows=ns,nrows=1,delimiter='=')[0].str.split()[0]
        nnlb = int(nnlb)

        bt.append(btype)
        label = ' '.join( pd.read_csv(hgrid,header=None,skiprows=ns,nrows=1,delimiter='=')[1].str.split()[0][-3:] )
        lp = pd.concat([lp,pd.read_csv(hgrid,header=None,skiprows=ns+1,nrows=nnlb,names=[label])], axis=1)

    lp.index.name = 'id' # set index name to match q,e
    lp.index = lp.index + 1 #  ''
        

    #DataFrame with the type of land boundary
    bbt = pd.DataFrame({'type':bt})
    bbt.index.name = 'id'
    bbt.index = bbt.index + 1
else:
    lp=pd.DataFrame({})
    bbt = pd.DataFrame({})

In [33]:
lp.head()

""


In [34]:
# merge to one xarray DataSet
g = xr.merge([q.to_xarray(), e.to_xarray(), op.to_xarray(), lp.to_xarray(), bbt.to_xarray()])


In [35]:
g

<xarray.Dataset>
Dimensions:  (id: 866550, index: 0)
Coordinates:
  * id       (id) int64 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 ...
  * index    (index) object 
Data variables:
    x        (id) float64 128.4 85.84 0.02595 90.03 179.9 -89.96 -135.0 ...
    y        (id) float64 89.98 -89.87 0.07616 -0.02094 0.1108 0.007103 ...
    z        (id) float64 4.112e+03 -2.775e+03 4.933e+03 4.197e+03 5.241e+03 ...
    nv       (id) int64 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 ...
    a        (id) int64 74458 386250 425016 65902 230673 386450 127287 ...
    b        (id) int64 76761 311077 375158 65903 191175 386449 220958 ...
    c        (id) int64 263429 410897 427834 65904 372416 386451 236361 ...